## Word - context matrix

word - context matrix 를 만들기 위하여 4 개의 문장을 샘플로 이용합니다. KoNLPy 의 Okt 를 tokenizer 로 이용합니다.

In [1]:
import config
import konlpy
from konlpy.tag import Okt

print('konlpy == {}'.format(konlpy.__version__))

soynlp=0.0.49
added lovit_textmining_dataset
konlpy == 0.5.1


네 개의 문장을 토크나이징합니다. 

In [2]:
sent = '역사적인 남북 정상 간 핫라인이 20일 청와대와 북한 국무위원회 사이에 정식 개통됐다'

tokenizer = Okt()
words = tokenizer.pos(sent, join=True)

In [3]:
from pprint import pprint
# pprint(sents)

windows = 2 일 때, 앞 뒤의 2 단어씩을 contexts 로 counting 합니다. defaultdict 를 이용하여 base2context = {base: {context:freq}} 를 만듭니다. 

In [4]:
from collections import defaultdict

windows = 2

# scanning (word, context) pairs
base2contexts = defaultdict(lambda: defaultdict(int))

# debug code
print(words, end='\n\n')

n = len(words)

for i, base in enumerate(words):

    b = max(0, i - windows)
    e = min(i + windows, n) + 1

    # left_contexts
    left_contexts = words[b:i]
    for context in left_contexts:
        base2contexts[base][context] += 1

    # right_contexts
    right_contexts = words[i+1:e]
    for context in right_contexts:
        base2contexts[base][context] += 1

    # debug code
    print('base = {}'.format(base))
    print('left = {}'.format(left_contexts), end='\n')
    print('right = {}'.format(right_contexts), end='\n\n')

['역사/Noun', '적/Suffix', '인/Josa', '남북/Noun', '정상/Noun', '간/Noun', '핫라인/Noun', '이/Josa', '20일/Number', '청와대/Noun', '와/Josa', '북한/Noun', '국무위원/Noun', '회/Noun', '사이/Noun', '에/Josa', '정식/Noun', '개통/Noun', '됐다/Verb']

base = 역사/Noun
left = []
right = ['적/Suffix', '인/Josa']

base = 적/Suffix
left = ['역사/Noun']
right = ['인/Josa', '남북/Noun']

base = 인/Josa
left = ['역사/Noun', '적/Suffix']
right = ['남북/Noun', '정상/Noun']

base = 남북/Noun
left = ['적/Suffix', '인/Josa']
right = ['정상/Noun', '간/Noun']

base = 정상/Noun
left = ['인/Josa', '남북/Noun']
right = ['간/Noun', '핫라인/Noun']

base = 간/Noun
left = ['남북/Noun', '정상/Noun']
right = ['핫라인/Noun', '이/Josa']

base = 핫라인/Noun
left = ['정상/Noun', '간/Noun']
right = ['이/Josa', '20일/Number']

base = 이/Josa
left = ['간/Noun', '핫라인/Noun']
right = ['20일/Number', '청와대/Noun']

base = 20일/Number
left = ['핫라인/Noun', '이/Josa']
right = ['청와대/Noun', '와/Josa']

base = 청와대/Noun
left = ['이/Josa', '20일/Number']
right = ['와/Josa', '북한/Noun']

base = 와/Josa
left = ['20일/Number', '청와대/

defaultdict 를 dict 로 변환하면 pprint 하여 보기가 편합니다. 

In [5]:
base2contexts = dict(base2contexts)

In [6]:
# pprint(base2contexts)

vocabulary 를 만듭니다. 

defaultdict 를 이용하면 자동으로 단어를 추가하는 dict 를 만들 수 있습니다. 

In [7]:
vocabulary = defaultdict(lambda: len(vocabulary))

for char in 'a b c b d'.split():
    print('{} = {}'.format(char, vocabulary[char]))

print(dict(vocabulary))

a = 0
b = 1
c = 2
b = 1
d = 3
{'a': 0, 'b': 1, 'c': 2, 'd': 3}


co-occurrence matrix 는 {row:{col:frequency}} 형식입니다. row 와 col 의 값을 vocabulary 에 넣어 index 로 만듭니다. 그리고 cooccurrence frequency 를 data 에 입력하여 csr_matrix 형식으로 만듭니다.

In [8]:
from scipy.sparse import csr_matrix

# Encoding dict to vectors
vocabulary = defaultdict(lambda: len(vocabulary))

rows = []
cols = []
data = []
for base, contexts in base2contexts.items():
    base_idx = vocabulary[base]
    for context, cooccurrence in contexts.items():
        context_idx = vocabulary[context]
        rows.append(base_idx)
        cols.append(context_idx)
        data.append(cooccurrence)

x = csr_matrix((data, (rows, cols)))
vocabulary = dict(vocabulary)

co-occurrence matrix 의 크기는 (19, 19) 이며

In [9]:
x.shape

(19, 19)

vocabulary 는 아래와 같습니다.

In [10]:
vocabulary

{'20일/Number': 8,
 '간/Noun': 5,
 '개통/Noun': 17,
 '국무위원/Noun': 12,
 '남북/Noun': 3,
 '됐다/Verb': 18,
 '북한/Noun': 11,
 '사이/Noun': 14,
 '에/Josa': 15,
 '역사/Noun': 0,
 '와/Josa': 10,
 '이/Josa': 7,
 '인/Josa': 2,
 '적/Suffix': 1,
 '정상/Noun': 4,
 '정식/Noun': 16,
 '청와대/Noun': 9,
 '핫라인/Noun': 6,
 '회/Noun': 13}

dense matrix 로 만들면 co-occurrence 를 확인할 수 있습니다.

In [11]:
x.todense()

matrix([[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0,

## using soynlp

이 과정은 soynlp 의 sent_to_word_contexts_matrix 를 이용할 수도 있습니다.

tokenizer 의 기본값은 lambda x:x.split() 입니다. 앞서 이용한 Okt 의 pos(join=True) 를 이용하기 위하여 custom_tokenize 함수를 만듭니다.

input 은 list of str 형식의 sentences 입니다. 이를 위해서 `sent` 를 list 로 감싸서 입력합니다.

In [12]:
from soynlp.vectorizer import sent_to_word_contexts_matrix

custom_tokenizer = lambda s:tokenizer.pos(s, join=True)

x, idx_to_vocab = sent_to_word_contexts_matrix(
    [sent],
    windows = 2,
    min_tf = 1,
    tokenizer = custom_tokenizer,
    verbose = True
)

Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.501 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.501 Gb
  - (word, context) matrix was constructed. shape = (19, 19)                    
  - done


동일한 크기의 행렬을 얻을 수 있습니다.

In [13]:
x.shape

(19, 19)

하지만 index 가 달라져서 matrix 의 모양은 다릅니다.

In [14]:
x.todense()

matrix([[0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
        [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
        [1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0],
        [1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0,

In [15]:
idx_to_vocab

['정상/Noun',
 '에/Josa',
 '적/Suffix',
 '인/Josa',
 '간/Noun',
 '20일/Number',
 '이/Josa',
 '정식/Noun',
 '핫라인/Noun',
 '와/Josa',
 '청와대/Noun',
 '국무위원/Noun',
 '회/Noun',
 '됐다/Verb',
 '북한/Noun',
 '사이/Noun',
 '개통/Noun',
 '역사/Noun',
 '남북/Noun']